In [ ]:
import kagglehub
from google.colab import files

# Upload kaggle.json (skip if already uploaded)
files.upload()  # Upload kaggle.json
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# Download dataset
path = kagglehub.dataset_download("sriramr/fruits-fresh-and-rotten-for-classification")
print("Path to dataset files:", path)

# Verify dataset structure
import os
for root, dirs, files in os.walk(path):
    print(f"Directory: {root}, Files: {len(files)}")

Saving kaggle.json to kaggle.json
Path to dataset files: /kaggle/input/fruits-fresh-and-rotten-for-classification
Directory: /kaggle/input/fruits-fresh-and-rotten-for-classification, Files: 0
Directory: /kaggle/input/fruits-fresh-and-rotten-for-classification/dataset, Files: 0
Directory: /kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/dataset, Files: 0
Directory: /kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/dataset/test, Files: 0
Directory: /kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/dataset/test/rottenbanana, Files: 530
Directory: /kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/dataset/test/freshoranges, Files: 388
Directory: /kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/dataset/test/rottenoranges, Files: 403
Directory: /kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/dataset/test/freshbanana, Files: 381
Directory: /kaggle/input/fruits-fresh-and-rotten-for-classification/datas

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

def organize_dataset(source_dir, target_dir, classes=['freshapples', 'rottenapples'], val_split=0.2):
    train_dir = os.path.join(target_dir, 'train')
    val_dir = os.path.join(target_dir, 'val')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    train_source = os.path.join(source_dir, 'dataset', 'train')
    if not os.path.exists(train_source):
        raise FileNotFoundError(f"Train directory not found at: {train_source}")

    for cls in classes:
        source_cls_dir = os.path.join(train_source, cls)
        if not os.path.exists(source_cls_dir):
            raise FileNotFoundError(f"Class directory not found: {source_cls_dir}")

        train_cls_dir = os.path.join(train_dir, cls)
        val_cls_dir = os.path.join(val_dir, cls)
        os.makedirs(train_cls_dir, exist_ok=True)
        os.makedirs(val_cls_dir, exist_ok=True)

        images = [os.path.join(source_cls_dir, f) for f in os.listdir(source_cls_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if not images:
            print(f"Warning: No images found in {source_cls_dir}")
            continue

        train_imgs, val_imgs = train_test_split(images, test_size=val_split, random_state=42)
        print(f"{cls}: {len(train_imgs)} train, {len(val_imgs)} val")

        for img in train_imgs:
            dest = os.path.join(train_cls_dir, os.path.basename(img))
            shutil.copy(img, dest)
        for img in val_imgs:
            dest = os.path.join(val_cls_dir, os.path.basename(img))
            shutil.copy(img, dest)

    print("Dataset organized into train and val splits")
    print(f"Train classes: {os.listdir(train_dir)}")
    print(f"Val classes: {os.listdir(val_dir)}")

organize_dataset('/kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/', 'dataset')

freshapples: 1354 train, 339 val
rottenapples: 1873 train, 469 val
Dataset organized into train and val splits
Train classes: ['freshapples', 'rottenapples']
Val classes: ['freshapples', 'rottenapples']


In [ ]:
import os
from PIL import Image

# Check directory contents
for root, dirs, files in os.walk('dataset'):
    print(f"Directory: {root}, Files: {len(files)}")

# Verify images
def verify_images(folder):
    for root, _, files in os.walk(folder):
        for file in files:
            try:
                img = Image.open(os.path.join(root, file))
                img.verify()
                img.close()
            except:
                print(f"Corrupted image: {file}")

verify_images('dataset')

Directory: dataset, Files: 0
Directory: dataset/train, Files: 0
Directory: dataset/train/freshapples, Files: 1354
Directory: dataset/train/rottenapples, Files: 1873
Directory: dataset/val, Files: 0
Directory: dataset/val/freshapples, Files: 339
Directory: dataset/val/rottenapples, Files: 469


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import time
from google.colab import files

# Clear GPU memory
torch.cuda.empty_cache()

# Define data directories
data_dir = 'dataset'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Load datasets
image_datasets = {
    'train': datasets.ImageFolder(train_dir, data_transforms['train']),
    'val': datasets.ImageFolder(val_dir, data_transforms['val'])
}
dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=2)
    for x in ['train', 'val']
}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(f"Classes: {class_names}")
if len(class_names) != 2:
    raise ValueError(f"Expected 2 classes, got {len(class_names)}: {class_names}")

# Load ResNet50
model = models.resnet50(weights='IMAGENET1K_V1')
for param in model.parameters():
    param.requires_grad = False  # Freeze base layers
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 128),
    nn.ReLU(),
    nn.Linear(128, 1),
    nn.Sigmoid()
)

# Move to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    start_time = time.time()
    for phase in ['train', 'val']:
        model.train() if phase == 'train' else model.eval()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            preds = (outputs > 0.5).float()
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print(f'Epoch time: {(time.time() - start_time)/60:.2f} minutes')

    # Save checkpoint
    os.makedirs('models', exist_ok=True)
    torch.save(model.state_dict(), f'models/spoilage_cnn_epoch_{epoch}.pth')
    files.download(f'models/spoilage_cnn_epoch_{epoch}.pth')

# Save final model
torch.save(model.state_dict(), 'models/spoilage_cnn.pth')
files.download('models/spoilage_cnn.pth')

Classes: ['freshapples', 'rottenapples']
Epoch 0/4
train Loss: 0.2117 Acc: 0.9154
val Loss: 0.0715 Acc: 0.9777
Epoch time: 0.48 minutes


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Epoch 1/4
train Loss: 0.1190 Acc: 0.9554
val Loss: 0.0511 Acc: 0.9790
Epoch time: 0.49 minutes


<IPython.core.display.Javascript object>

...

Epoch 4/4
train Loss: 0.1034 Acc: 0.9603
val Loss: 0.0492 Acc: 0.9827
Epoch time: 0.47 minutes


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch
from torchvision import models

model = models.resnet50(weights=None)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(model.fc.in_features, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 1),
    torch.nn.Sigmoid()
)
model.load_state_dict(torch.load('models/spoilage_cnn.pth'))
print("Model loaded successfully")

Model loaded successfully


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from sklearn.metrics import precision_score, recall_score, f1_score
import os
import numpy as np

# Define data directories
data_dir = 'dataset'
val_dir = os.path.join(data_dir, 'val')

# Data transformations
data_transforms = {
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Load validation dataset
val_dataset = datasets.ImageFolder(val_dir, data_transforms['val'])
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)
dataset_size = len(val_dataset)
class_names = val_dataset.classes
print(f"Classes: {class_names}")

# Load model
model = models.resnet50(weights=None)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),
    nn.Linear(128, 1),
    nn.Sigmoid()
)
model.load_state_dict(torch.load('models/spoilage_cnn.pth'))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# Evaluate
all_preds = []
all_labels = []
running_corrects = 0
running_loss = 0.0
criterion = nn.BCELoss()

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        preds = (outputs > 0.5).float()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        all_preds.extend(preds.cpu().numpy().flatten())
        all_labels.extend(labels.cpu().numpy().flatten())

# Compute metrics
val_loss = running_loss / dataset_size
val_acc = running_corrects.double() / dataset_size
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_acc:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')

Classes: ['freshapples', 'rottenapples']
Validation Loss: 0.0492
Validation Accuracy: 0.9827
Precision: 0.9730
Recall: 0.9979
F1-Score: 0.9853


In [ ]:
from PIL import Image
import torch
from torchvision import models, transforms
import os

# Load model
model = models.resnet50(weights=None)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(model.fc.in_features, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 1),
    torch.nn.Sigmoid()
)
model.load_state_dict(torch.load('models/spoilage_cnn.pth'))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# Transform for single image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Test on a sample image
sample_image_path = '/kaggle/apple-1834639_1280.jpg'  # Replace with a valid image path
image = Image.open(sample_image_path).convert('RGB')
image = transform(image).unsqueeze(0).to(device)

with torch.no_grad():
    output = model(image)
    pred = output.item()
    label = 'rottenapples' if pred > 0.7 else 'freshapples'
    print(f'Prediction: {label}, Confidence: {pred:.4f}')

Prediction: freshapples, Confidence: 0.5975


In [ ]:
pip install fastapi uvicorn python-multipart torch torchvision pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from fastapi import FastAPI, File, UploadFile, HTTPException
from PIL import Image
import torch
from torchvision import models, transforms
import io

app = FastAPI()

# Load model
model = models.resnet50(weights=None)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(model.fc.in_features, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 1),
    torch.nn.Sigmoid()
)
model.load_state_dict(torch.load('models/spoilage_cnn.pth'))
model.eval()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Transform for input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    if not file.content_type.startswith('image/'):
        raise HTTPException(status_code=400, detail="Invalid file type. Please upload an image.")

    # Read and process image
    contents = await file.read()
    image = Image.open(io.BytesIO(contents)).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    # Predict
    with torch.no_grad():
        output = model(image)
        pred = output.item()
        label = 'rottenapples' if pred > 0.7 else 'freshapples'

    return {"prediction": label, "confidence": pred}

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Create YOLO dataset structure
os.makedirs('yolo_dataset/images/train', exist_ok=True)
os.makedirs('yolo_dataset/images/val', exist_ok=True)
os.makedirs('yolo_dataset/labels/train', exist_ok=True)
os.makedirs('yolo_dataset/labels/val', exist_ok=True)

# Copy images from existing dataset
for split in ['train', 'val']:
    for cls in ['freshapples', 'rottenapples']:
        src_dir = f'dataset/{split}/{cls}'
        dst_dir = f'yolo_dataset/images/{split}'
        if os.path.exists(src_dir):
            for img in os.listdir(src_dir):
                if img.lower().endswith(('.jpg', '.jpeg', '.png')):
                    shutil.copy(os.path.join(src_dir, img), os.path.join(dst_dir, img))

# Generate placeholder annotations (class_id=0 for apple, centered box)
for split in ['train', 'val']:
    img_dir = f'yolo_dataset/images/{split}'
    label_dir = f'yolo_dataset/labels/{split}'
    for img in os.listdir(img_dir):
        if img.lower().endswith(('.jpg', '.jpeg', '.png')):
            # Placeholder: Assume apple is centered, 50% of image size
            with open(os.path.join(label_dir, img.replace('.jpg', '.txt').replace('.png', '.txt')), 'w') as f:
                f.write('0 0.5 0.5 0.5 0.5\n')  # class_id x_center y_center width height

# Create data.yaml
with open('yolo_dataset/data.yaml', 'w') as f:
    f.write("""
train: ./yolo_dataset/images/train
val: ./yolo_dataset/images/val
nc: 1
names: ['apple']
""")

# Verify dataset
for split in ['train', 'val']:
    img_dir = f'yolo_dataset/images/{split}'
    label_dir = f'yolo_dataset/labels/{split}'
    print(f"{split} images: {len(os.listdir(img_dir)) if os.path.exists(img_dir) else 0}")
    print(f"{split} labels: {len(os.listdir(label_dir)) if os.path.exists(label_dir) else 0}")

train images: 3227
train labels: 3227
val images: 808
val labels: 808


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image

# Clear any existing yolo_dataset
shutil.rmtree('yolo_dataset', ignore_errors=True)

# Create YOLO dataset structure
os.makedirs('yolo_dataset/images/train', exist_ok=True)
os.makedirs('yolo_dataset/images/val', exist_ok=True)
os.makedirs('yolo_dataset/labels/train', exist_ok=True)
os.makedirs('yolo_dataset/labels/val', exist_ok=True)

# Copy images from dataset/train and dataset/val
for split in ['train', 'val']:
    for cls in ['freshapples', 'rottenapples']:
        src_dir = f'dataset/{split}/{cls}'
        dst_dir = f'yolo_dataset/images/{split}'
        if os.path.exists(src_dir):
            for img in os.listdir(src_dir):
                if img.lower().endswith(('.jpg', '.jpeg', '.png')):
                    shutil.copy(os.path.join(src_dir, img), os.path.join(dst_dir, img))
        else:
            print(f"Warning: {src_dir} not found")

# Generate placeholder annotations (class_id=0 for apple)
for split in ['train', 'val']:
    img_dir = f'yolo_dataset/images/{split}'
    label_dir = f'yolo_dataset/labels/{split}'
    if os.path.exists(img_dir):
        for img in os.listdir(img_dir):
            if img.lower().endswith(('.jpg', '.jpeg', '.png')):
                try:
                    # Verify image
                    with Image.open(os.path.join(img_dir, img)) as pil_img:
                        pil_img.verify()
                    # Create label file: centered 50% box
                    label_path = os.path.join(label_dir, img.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt'))
                    with open(label_path, 'w') as f:
                        f.write('0 0.5 0.5 0.5 0.5\n')  # class_id x_center y_center width height
                except:
                    print(f"Skipping corrupted image: {img}")
    else:
        print(f"Error: {img_dir} not found")

# Create data.yaml
with open('yolo_dataset/data.yaml', 'w') as f:
    f.write("""
train: /content/yolo_dataset/images/train
val: /content/yolo_dataset/images/val
nc: 1
names: ['apple']
""")

# Verify dataset
for split in ['train', 'val']:
    img_dir = f'yolo_dataset/images/{split}'
    label_dir = f'yolo_dataset/labels/{split}'
    img_count = len(os.listdir(img_dir)) if os.path.exists(img_dir) else 0
    label_count = len(os.listdir(label_dir)) if os.path.exists(label_dir) else 0
    print(f"{split} images: {img_count}")
    print(f"{split} labels: {label_count}")
    if img_count != label_count:
        print(f"Warning: Mismatch in {split} - images: {img_count}, labels: {label_count}")

train images: 3227
train labels: 3227
val images: 808
val labels: 808


In [ ]:
# !pip install ultralytics

from ultralytics import YOLO
from google.colab import files
import os

# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # Pre-trained YOLOv8 nano

# Train
model.train(
    data='yolo_dataset/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    device=0,  # Use GPU
    project='runs/train',
    name='exp'
)

# Save model
os.makedirs('models', exist_ok=True)
model.save('models/yolo_apple.pt')
files.download('models/yolo_apple.pt')

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, pro

100%|██████████| 755k/755k [00:00<00:00, 108MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 284MB/s]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/torch_utils.py:271: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:217.)
  fusedconv.weight.copy_(torch.mm(w_bn, w_conv).view(fusedconv.weight.shape))
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/torch_utils.py:280: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Co

...

Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:11<00:00,  2.35it/s]


                   all        808        808          1          1      0.995      0.994
Speed: 0.3ms preprocess, 4.8ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/train/exp2


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>